In [15]:
# all import statements needed for the project, for example:

import os

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
from bs4 import BeautifulSoup
import re
from datetime import datetime
from typing import List

In [91]:
TAXI_URL: str = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Date range for filtering
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2024, 3, 31)

def get_taxi_html() -> str:
    """Fetch the HTML content of the taxi data page."""
    response = requests.get(TAXI_URL)
    response.raise_for_status()
    html = response.content
    return html

def find_taxi_parquet_links() -> List[str]:
    """Find links to Yellow Taxi and HVFHV Parquet files within the date range."""
    html = get_taxi_html()
    soup = bs4.BeautifulSoup(html, "html.parser")
    
    # Find all <a> tags with relevant titles
    yellow_a_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    hvfhv_a_tags = soup.find_all("a", href=re.compile(r"fhvhv.*\.parquet", re.IGNORECASE))
    
    # Combine all links
    all_a_tags = yellow_a_tags + hvfhv_a_tags
    
    # Extract href attributes and filter based on .parquet
    parquet_links = [a["href"].strip() for a in all_a_tags if ".parquet" in (a.get("href") or "")]
    return filter_links_by_date(parquet_links)

def filter_links_by_date(links: List[str]) -> List[str]:
    """Filter Parquet file links by date, retaining only those within the specified range."""
    filtered_links = []
    date_pattern = re.compile(r"_(\d{4})-(\d{2})\.parquet")
    
    for link in links:
        match = date_pattern.search(link)
        if match:
            year, month = int(match.group(1)), int(match.group(2))
            file_date = datetime(year, month, 1)
            if START_DATE <= file_date <= END_DATE:
                filtered_links.append(link)
    
    return filtered_links

def download_files(links: List[str], folder_name: str) -> None:
    """Download files from a list of links and save them to the specified folder."""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    
    for link in links:
        file_name = link.split("/")[-1]
        file_path = os.path.join(folder_name, file_name)
        print(f"Downloading {file_name} from {link}...")
        
        # Request with headers to mimic a browser
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Check if download was successful
        
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"Downloaded {file_name}")

# Find and download filtered links
filtered_links = find_taxi_parquet_links()
download_files(filtered_links, "taxi_data")

Downloaded yellow_tripdata_2024-01.parquet
Downloaded yellow_tripdata_2024-02.parquet
Downloaded yellow_tripdata_2024-03.parquet
Downloaded fhvhv_tripdata_2024-01.parquet
Downloaded fhvhv_tripdata_2024-02.parquet
Downloaded fhvhv_tripdata_2024-03.parquet


In [93]:
import pandas as pd

# Load the dataset
df = pd.read_parquet(r"C:\Users\sanch\TOOLS for Analytics\Project 1\taxi_data\fhvhv_tripdata_2024-01.parquet")

# Display the first few rows of the DataFrame
print(df.head())

# Display column names to understand the structure
print(df.columns)

  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B03404               B03404   
1            HV0003               B03404               B03404   
2            HV0003               B03404               B03404   
3            HV0003               B03404               B03404   
4            HV0003               B03404               B03404   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2024-01-01 00:21:47 2024-01-01 00:25:06 2024-01-01 00:28:08   
1 2024-01-01 00:10:56 2024-01-01 00:11:08 2024-01-01 00:12:53   
2 2024-01-01 00:20:04 2024-01-01 00:21:51 2024-01-01 00:23:05   
3 2024-01-01 00:35:46 2024-01-01 00:39:59 2024-01-01 00:41:04   
4 2024-01-01 00:48:19 2024-01-01 00:56:23 2024-01-01 00:57:21   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  sales_tax  \
0 2024-01-01 01:05:39           161           158        2.83  ...       4.05   
1 2024-01-01 00:20:05           137            79       

In [95]:
# check how many rows before Uber
print(len(df))

19663930


In [99]:
# Define the folder path containing your Parquet files
folder_path = r"C:\Users\sanch\TOOLS for Analytics\Project 1\taxi_data"

# Define the regex pattern to find files that contain "fhvhv" and end with ".parquet"
pattern = re.compile(r".*fhvhv.*\.parquet$", re.IGNORECASE)

# List all the Parquet files that match the pattern
parquet_files = [f for f in os.listdir(folder_path) if pattern.match(f)]

# Display the filtered file paths
print(parquet_files)

['fhvhv_tripdata_2024-01.parquet', 'fhvhv_tripdata_2024-02.parquet', 'fhvhv_tripdata_2024-03.parquet']


In [89]:
# check wether there is Non Uber Data in hour folder
df = pd.read_parquet(r"C:\Users\sanch\TOOLS for Analytics\Project 1\taxi_data\fhvhv_tripdata_2024-01.parquet")

# Display the first few rows of the DataFrame
print(df.head())

# Display column names to understand the structure
print(df.columns)

Empty DataFrame
Columns: [hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag]
Index: []

[0 rows x 24 columns]
Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')


In [ ]:

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = ""
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [1]:
# hi hi hi 